### Домашнее задание №3 к лекции «Функции и работа с данными»

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import os

#### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [2]:
task_1_files = [file for file in os.listdir('ml-latest') if '.csv' in file]
# распакованный архив с этого сайта кладем в папку с ноутбуком
task_1_files

['links.csv', 'tags.csv', 'ratings.csv', 'keywords.csv', 'movies.csv']

In [3]:
ratings = pd.read_csv(os.getcwd() + '/ml-latest/' + task_1_files[2])

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
%%time
ratings['class_1'] = ratings['rating'].apply(
    lambda x: 'низкий рейтинг' if x <= 2 else('средний рейтинг' if x <= 4 else'высокий рейтинг'))

CPU times: user 21.6 ms, sys: 1.41 ms, total: 23 ms
Wall time: 21.8 ms


In [6]:
ratings.groupby('class_1').agg({'rating': 'unique'})

,rating
class_1,
высокий рейтинг,"[5.0, 4.5]"
низкий рейтинг,"[2.0, 1.0, 1.5, 0.5]"
средний рейтинг,"[2.5, 3.0, 4.0, 3.5]"


In [7]:
def get_class(row):
    if row.rating <= 2:
        return 'низкий рейтинг'
    elif row.rating <= 4:
        return 'средний рейтинг'
    else:
        return 'высокий рейтинг'

In [8]:
%%time
ratings['class_2'] = ratings.apply(get_class, axis=1)

CPU times: user 1.5 s, sys: 11.2 ms, total: 1.51 s
Wall time: 1.51 s


In [9]:
ratings.groupby('class_2').agg({'rating': 'unique'})

,rating
class_2,
высокий рейтинг,"[5.0, 4.5]"
низкий рейтинг,"[2.0, 1.0, 1.5, 0.5]"
средний рейтинг,"[2.5, 3.0, 4.0, 3.5]"


Результат одинаков. Первый вариант быстрее, что объясняется только тем что метод `apply` обращается к конкретному столбцу, а второй ко всей строке

#### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.  
Результат классификации запишите в отдельный столбец region.

In [10]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [11]:
keywords = pd.read_csv(os.getcwd() + '/ml-latest/' + task_1_files[3])

In [12]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   keyword  100000 non-null  object
 1   shows    100000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [13]:
def get_region(keyword):
    for i,v in geo_data.items():
        if keyword in v:
            return i
    return 'undefined'

In [14]:
keywords['region'] = keywords['keyword'].apply(get_region)

In [15]:
keywords.groupby('region').agg({'keyword': 'unique'})

,keyword
region,
undefined,"[вк, одноклассники, порно, ютуб, вконтакте, од..."
Дальний Восток,"[владивосток, хабаровск, сахалин]"
Северо-Запад,"[мурманск, псков, петербург]"
Центр,"[москва, ярославль, тула]"


Формула вроде бы отработала правильно. Проверим сколько ключевых слов попало в __undefined__

In [16]:
len(keywords.groupby('region').agg({'keyword': 'unique'}).loc['undefined', 'keyword'])

99991

Из 100000 слов 99991. Каждое слово по разу. Выведем кратенький датафрейм только с городами в качестве ключевых слов

In [17]:
keywords[keywords['keyword'].isin(reduce(lambda x,y: x + y, list(geo_data.values())))]

,keyword,shows,region
1259,москва,153050,Центр
5858,владивосток,43038,Дальний Восток
6075,ярославль,41720,Центр
8977,мурманск,29984,Северо-Запад
9084,хабаровск,29655,Дальний Восток
11222,псков,24885,Северо-Запад
11785,тула,23898,Центр
20823,петербург,14859,Северо-Запад
21445,сахалин,14494,Дальний Восток


#### Задание 3
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

P/S/ все файлы находятся в архиве «дополнительные файлы к лекции и дз»

In [18]:
movies = pd.read_csv(os.getcwd() + '/ml-latest/' + task_1_files[4])
ratings = pd.read_csv(os.getcwd() + '/ml-latest/' + task_1_files[2])

In [19]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [21]:
def production_year(title):
    for i in list(range(1950,2011)):
        if str(i) in title:
            return i
    return 1900

In [22]:
movies['year'] = movies['title'].apply(production_year)

In [23]:
movies.set_index('movieId', inplace=True)

In [24]:
movies.sample(10)

,title,genres,year
movieId,,,
81641,Fair Game (2010),Drama|Thriller,2010
1874,Still Breathing (1997),Comedy|Romance,1997
4443,Outland (1981),Action|Sci-Fi|Thriller,1981
100527,Safe Haven (2013),Drama|Mystery|Romance,1900
106540,Delivery Man (2013),Comedy,1900
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,1977
61026,Red Cliff (Chi bi) (2008),Action|Adventure|Drama|War,2008
2,Jumanji (1995),Adventure|Children|Fantasy,1995
99106,"Guilt Trip, The (2012)",Comedy,1900


Похоже годы отразились согласно задуманному

В таблице __ratings__ определим средний рейтинг по каждому __movieId__. Полученный результат присоединим к таблице __movies__ по ключу __movieId__. после чего сгруппируем полученный датафрейм по годам и выведем средний рейтинг по году

In [25]:
result = movies.merge(
    ratings.groupby('movieId').agg({'rating':'mean'}), 
    how='left', left_index=True, right_index=True
).groupby('year').agg({'rating':'mean'}).loc[1950:2010,]

In [26]:
result.sort_values(by='rating', ascending=False)

,rating
year,
1951,3.906653
1952,3.802318
1962,3.772962
1953,3.738878
1957,3.693322
...,...
2002,3.191918
1997,3.183169
1990,3.166759
